<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-pinecone pinecone-notebooks openparse[ml] html2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 837.7 kB/s eta 0:00:00
   ━━━━

## Configuração de Variaveis

In [2]:
from google.colab import userdata

class Settings:
  HF_TOKEN          = userdata.get('HF_TOKEN')
  PINECONE_API_KEY  = userdata.get('PINECONE_API_KEY')
  GROQ_API_KEY      = userdata.get('GROQ_API_KEY')


settings = Settings()

## Pré-processamento de dados

### Download do dataset

In [ ]:
!mkdir -p data
!wget https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico -O "data/ppc_2018.pdf"

--2024-08-26 01:58:39--  https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico
Resolving www.uffs.edu.br (www.uffs.edu.br)... 200.135.49.107
Connecting to www.uffs.edu.br (www.uffs.edu.br)|200.135.49.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1383621 (1.3M) [application/pdf]
Saving to: ‘data/ppc_2018.pdf’

data/ppc_2018.pdf   100%[===================>]   1.32M   607KB/s    in 2.2s    

2024-08-26 01:58:43 (607 KB/s) - ‘data/ppc_2018.pdf’ saved [1383621/1383621]



In [ ]:
from google.colab import files

uploaded = files.upload()

Saving PPC_ABRIL-22(nova versão)-v35.pdf to PPC_ABRIL-22(nova versão)-v35 (1).pdf




### Transformação PDF -> HTML -> Markdown




In [ ]:
import openparse
import html2text
from langchain_core.documents import Document


def parse_document_to_documents(file_path, source_name):
    # Parse the document using OpenParse
    parser = openparse.DocumentParser(table_args={"parsing_algorithm": "pymupdf"})
    document = parser.parse(file_path)

    documents = []

    # Iterate over the nodes and convert each to a Document
    for node in document.nodes:
        html_content = node.text

        if html_content:
            document_chunk = Document(
                metadata={'source': source_name},
                page_content=html_content
            )
            documents.append(document_chunk)

    return documents


In [ ]:
ppc_2018_chunks = parse_document_to_documents('data/ppc_2018.pdf', 'ppc2018')

In [ ]:
ppc_2024_chunks = parse_document_to_documents('PPC_ABRIL-22(nova versão)-v35.pdf', 'ppc2024')

### Geração de embeddings e indíces do banco vetorial

In [3]:
class EmbeddingModelSpecs:
  def __init__(self):
    self.name      = 'sentence-transformers/distiluse-base-multilingual-cased-v1'
    self.dimension = 512

embeddings_model = EmbeddingModelSpecs()


In [4]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=settings.PINECONE_API_KEY)

In [5]:
import time

index_name = "index-ppc-2018-markdown"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=embeddings_model.dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [6]:
!pip install -qU langchain-huggingface langchain langchain-community unstructured==0.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.3 MB/s eta 0:00:00


In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name=embeddings_model.name)

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [8]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from uuid import uuid4
from langchain_core.documents import Document
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_documents(file_path):
    """Load documents from a Markdown file using UnstructuredMarkdownLoader."""
    # Usar um loader específico para Markdown
    loader = UnstructuredMarkdownLoader(file_path)
    documents = loader.load()
    return documents

def split_documents_into_chunks(documents, chunk_size=2000, chunk_overlap=400):
    """Split the loaded documents into text chunks based on Markdown structure."""
    # Usando RecursiveCharacterTextSplitter para respeitar a estrutura do Markdown
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n# ", "\n## ", "\n### ", "\n", " "]
    )
    texts = text_splitter.split_documents(documents)
    return texts

def create_documents_from_chunks(texts, source_name):
    """Create Document objects from text chunks."""
    documents = [
        Document(page_content=text.page_content, metadata={"source": source_name}) for text in texts
    ]
    return documents

def generate_unique_ids(num_ids):
    """Generate a list of unique UUIDs."""
    return [str(uuid4()) for _ in range(num_ids)]

def add_documents_to_vector_store(vector_store, documents, uuids):
    """Add documents with unique IDs to the vector store."""
    print(vector_store.add_documents(documents=documents, ids=uuids))


### Ingestão PPC 2018

In [ ]:
# file_path = "data/ppc_2018.md"
# text = load_documents(file_path)
# chunks = split_documents_into_chunks(text)
# documents = create_documents_from_chunks(chunks, source_name="ppc2018.md")


In [ ]:
uuids = generate_unique_ids(len(ppc_2018_chunks))
add_documents_to_vector_store(vector_store, ppc_2018_chunks, uuids)

['5af798da-454e-4672-bca8-33bc9eacec75', 'bacfeaed-6f2e-463a-8d31-2caf1229da2b', '26b6b9e9-2ab4-44b9-8e84-32d20d4a6e18', '4a66c938-0dd4-43ee-a41e-db1d77e16b46', 'bbecd110-04b4-4892-a857-dd93fcdcfc4c', '316fa45b-29a1-4b0f-b542-b6daa56bb859', '6ca7f2d3-4b32-4d26-9f19-95b588714a4a', '5a16f5d2-023b-4fe4-a32f-6aabde46b577', 'e29f27c5-4520-4f89-aed2-e65245c721ae', '2f4f5eb7-a21b-4a67-a017-4610cebb8873', 'c7795b1d-eb95-46b8-a3bf-d28af5d21928', 'a42639a8-4172-4d02-b621-3207b7da59f6', '216e24d9-0fd6-4c53-9c65-c3c988a115f5', '493b3e1f-f3b3-47bf-8fd3-88746bdc1233', 'e41a4044-0568-4d21-b33c-2bb3976e04f4', '47f26769-e777-490a-8c63-c18da3b7fc59', '4d6fe5a4-7853-4e7a-ad7c-f7d2175d8ed9', 'c4758b09-e97e-4f6f-94b8-864e5b26979c', '1130529a-2049-4aef-b68a-bf2fbfca3a88', '5010b129-0e92-42cb-966d-6962705d54de', '834d40a8-fb57-4d3b-8b21-a9cdff232b7e', 'f745457b-6063-4db1-b81b-68cb28dfb2fc', 'da64f738-10f5-4a25-9207-b4c31cf71afd', 'dd8972d5-91c7-4da8-8848-9594470eac46', 'c103cb17-5fd9-4795-9765-33c06fb37b13',

### Ingestão PPC 2024

In [ ]:
# file_path = "data/ppc_2024.md"
# text = load_documents(file_path)
# chunks = split_documents_into_chunks(text)
# documents = create_documents_from_chunks(chunks, source_name="ppc2024.md")


In [ ]:
uuids = generate_unique_ids(len(ppc_2024_chunks))
add_documents_to_vector_store(vector_store, ppc_2024_chunks, uuids)

['5d02697c-12ba-4daa-812e-04a37b8d6d14', 'af8913ab-3bb0-40e5-9bb4-9597d8b34cda', 'd58da9c6-8a32-4b13-81f2-0dc8845d2fbd', '7c13f52e-50a6-46cc-9c6f-e41880bd9a5d', 'bc2dcf2f-c4d8-4ff8-95d3-da75cb42a6ed', 'ec1c2c30-bf95-43b8-8aca-7e08b63ef0cf', '0e9dbd97-b293-4c08-ba51-46c22f877ab0', 'f8a4dee6-a70c-4203-b825-9cc87fabbe61', '07fee48a-97c8-4aba-90f2-0698cb4c0e65', '49040a66-9d20-4bc8-be4a-be4adf908442', '1db701a5-fc86-4a48-b412-e0e6a27eada3', 'eb086141-56d1-4e84-86b4-bfc79cdbe5e3', 'df0eff59-8788-430c-aed7-7a4f304501e6', '0f915bd8-bb8b-4c4f-90d8-19eefa17b0d6', '55a1f916-794f-47ba-8a88-979f04a2ec0f', 'bd6fdccc-eb1e-4c7a-98cc-1865c8b463f9', '203916ac-6713-4c7a-af39-637987cdf7c8', '3eef49ac-47c2-4d31-8ee6-01fc7e1f5607', 'e4986dc9-3f4f-42a5-b3de-03aee286ac75', 'd51dd6b4-3419-4858-a58d-249d1bce9288', '66a23e4e-66de-43d4-a71b-55b7960ea956', 'dde35c18-2c3f-49a4-9435-adb2c4a543af', 'ffeb5b71-32c4-4dd7-8ae3-bb9a29d5f3c4', '85d04691-60bd-4bd1-a582-44847ac05908', 'd10dd0e8-a7e5-46de-ae5d-f11cee3f8ac1',

## Geração da resposta

In [9]:
!pip install -U langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.3 MB/s eta 0:00:00


In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq



llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    groq_api_key=settings.GROQ_API_KEY,
    max_retries=2
)



def query_expansion(query):

    query_expansion_llm = ChatGroq(
        model="gemma2-9b-it",
        temperature=0,
        groq_api_key=settings.GROQ_API_KEY,
        max_retries=2
    )

    prompt = f"""Esse é minha pergunta: {query}
    Me retorne uma string com 5 jeitos de escrever essa mesma pergunta em portugues.
    Considere um contexto academico de uma universidade.
    Retorne apenas as strings separadas com virgula sem nenhuma frase introdutória.
    """

    # Criando o prompt para a consulta expandida
    prompt_template = ChatPromptTemplate.from_messages([("human", prompt)])

    # Gerando a lista de consultas expandidas
    chain = prompt_template | query_expansion_llm
    response = chain.invoke({})
    return response.content


def get_context(query, source_name = 'ppc2024.md'):

    results = vector_store.search(
        query,
        "similarity",
        k=5,
        filter={"source": source_name},
    )


    return  '\n'.join([result.page_content for result in results])




def build_prompt(contexto, pergunta):
    # Cria o prompt para o contexto do PPC
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Você é um assistente especializado no Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS. "
                "Sua tarefa é fornecer respostas precisas sobre qualquer questão relacionada ao PPC, utilizando o contexto fornecido para guiar sua resposta."
                "Utilize apenas o contexto informado como referencia"
                "Caso a resposta não está no contexto, informe que não sabe a resposta e recomende consultar o documento original."

                "Responda exclusivamente em portugues brasileiro."
            ),
            (
            "human",
            f"Dado o contexto: {contexto}\n\nPergunta: {pergunta}. Utilize apenas o contexto fornecido para responder"),
        ]
    )
    return prompt

def get_completion(prompt):
    chain = prompt | llm

    response = chain.invoke({})
    return response


def get_answer(query, source_name):
    expanded_query = query_expansion(query)
    print(expanded_query)
    context = get_context(query, source_name)
    prompt = build_prompt(context, query)
    response = get_completion(prompt)

    return context, response.content


In [18]:
query = "Qual é o limite de horas para Atividades de Extensão?"
context, answer = get_answer(query, "ppc2024")
answer

Qual o limite de horas para atividades de extensão? ,  
Existe um limite de horas para atividades de extensão? , 
Quantas horas são permitidas para atividades de extensão? ,
Até quantas horas de atividades de extensão podem ser realizadas? ,
Qual a carga horária máxima para atividades de extensão? 



'O limite de horas para Atividades de Extensão é de 260 horas, conforme especificado no contexto fornecido. Essas horas incluem as atividades do Curso Complementar Obrigatório (CCR) de Introdução à Prática da Extensão e as atividades curriculares de extensão (ACEs). Além disso, podem ser ofertados CCRs optativos com carga horária em extensão para a integralização das atividades.'

In [ ]:
context

'participação ativa na aplicação e no acompanhamento do Projeto Pedagógico. <br><br>O Projeto Pedagógico do Curso prevê conteúdos, práticas, atividades de extensão e de <br><br>895  pesquisa implementados por meio de componentes curriculares obrigatórios e optativos, <br><br>sendo ministrados em salas de aulas e em laboratórios. O discente deve colocar em prática os <br><br>conteúdos através de projetos relacionados aos componentes, atividades curriculares <br><br>complementares, atividades de extensão e trabalho de conclusão de curso. <br><br>A organização curricular obedecerá aos princípios pedagógicos, metodológicos e <br><br>900  epistemológicos da proposta curricular da UFFS, tendo como subsídio as orientações <br><br>apresentadas no Projeto Pedagógico Institucional (PPI). \n1255  *8.7.2 Atividades curriculares complementares (Normatização no ANEXO I)*<br><br>As Atividades Curriculares Complementares (ACCs) constituem ações que visam a <br><br>complementação do processo ensino-apr

In [ ]:
results = vector_store.similarity_search(
    "Limite de Atividades de ensino",
    k=2,
    filter={"source": "ppc2024"},
)
results

[Document(metadata={'source': 'ppc2024'}, page_content='<table border="1">\n<tr><th>II</th><th>Atividades de Ensino (Limite horas: 120)</th><th></th><th></th><th></th></tr>\n<tr><td>II</td><td>Atividades de Ensino (Limite horas: 120)</td><td> </td><td> </td><td> </td></tr>\n<tr><td>Categoria</td><td>Descrição da Atividade</td><td>Limite de\nHoras</td><td>Horas por\nEvento</td><td>Documento\ncomprobatório</td></tr>\n<tr><td>2.1</td><td>Programas de monitoria</td><td>100\nDRAFT</td><td>total de horas\nde participação</td><td>Certificado, ou declaração\ndo responsável pelo\nprograma.</td></tr>\n<tr><td>2.2</td><td>Participação em grupos de\nestudos formais do curso de\nCiência da Computação da\nUFFS</td><td>60</td><td>total de horas\nde participação</td><td>Certificado, ou declaração\ndo docente responsável\npelo grupo.</td></tr>\n<tr><td>III</td><td>Outras Atividades Complementares (Limite horas: 120)</td><td> </td><td> </td><td> </td></tr>\n<tr><td>Categoria</td><td>Descrição da Ativida